# Mount S3 Bucket
This section contains code to mount an S3 bucket to the Databrick and securely handle AWS credentials

In [0]:

from pyspark.sql.functions import *
import urllib
from pyspark.sql.functions import col
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import to_timestamp

In [0]:
# Define the path to the Delta table containing AWS credentials
delta_table_path = "dbfs:/user/hive/warehouse/authentication_credentials"

# Read the Delta table to a Spark DataFrame
aws_keys_df = spark.read.format("delta").load(delta_table_path)

In [0]:
ACCESS_KEY = aws_keys_df.select('Access key ID').collect()[0]['Access key ID']
SECRET_KEY = aws_keys_df.select('Secret access key').collect()[0]['Secret access key']

# Encode the secret key for security
ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY, safe="")

In [0]:
# AWS S3 bucket name
AWS_S3_BUCKET = "user-0e2685691ff5-bucket"
# Mount name for the bucket
MOUNT_NAME = "/mnt/user-0e2685691ff5-bucket"
# Source url
SOURCE_URL = "s3n://{0}:{1}@{2}".format(ACCESS_KEY, ENCODED_SECRET_KEY, AWS_S3_BUCKET)
# Check if the mount point already exists before mounting
if not any(mount.mountPoint == MOUNT_NAME for mount in dbutils.fs.mounts()):
    dbutils.fs.mount(SOURCE_URL, MOUNT_NAME)
else:
    print(f"{MOUNT_NAME} is already mounted.")


/mnt/user-0e2685691ff5-bucket is already mounted.

# Process Data from S3 Topics
This section has function that reads in data from the three topics `pin`, `geo` and `user` from the mounted S3 bucket

In [0]:
# Content of the S3 bucket
display(dbutils.fs.ls("/mnt/user-0e2685691ff5-bucket/topics"))

path,name,size,modificationTime
dbfs:/mnt/user-0e2685691ff5-bucket/topics/0e2685691ff5.geo/,0e2685691ff5.geo/,0,1724559354084
dbfs:/mnt/user-0e2685691ff5-bucket/topics/0e2685691ff5.pin/,0e2685691ff5.pin/,0,1724559354084
dbfs:/mnt/user-0e2685691ff5-bucket/topics/0e2685691ff5.user/,0e2685691ff5.user/,0,1724559354084


In [0]:
# List of topics
list_of_topics = [".pin", ".geo", ".user"]

# read data from topic and return as a DataFrame
def read_topic_to_df(topic):
    # Path to files in the topic
    file_location = f"/mnt/user-0e2685691ff5-bucket/topics/0e2685691ff5{topic}/partition=0/*.json"
    # Read in JSONs from the mounted S3 bucket with schema inference
    df = spark.read.format("json").option("inferSchema", "true").load(file_location)
    return df


In [0]:
# DataFrames for each topic
df_pin = read_topic_to_df(".pin")
df_geo = read_topic_to_df(".geo")
df_user = read_topic_to_df(".user")


In [0]:
print("Displaying DataFrame for topic: pin")
display(df_pin)

Displaying DataFrame for topic: pin

category,description,downloaded,follower_count,image_src,index,is_image_or_video,poster_name,save_location,tag_list,title,unique_id
home-decor,"Традиционные шведские коттеджи, обычно с красным фасадом — это настоящее воплощением идеального зимнего уюта. Они обычно оформлены очень просто и ✌PUFIK. Beautiful Interiors. On…",1,136k,https://i.pinimg.com/originals/32/eb/72/32eb72e4fd8654c115a64528bd1f34b4.png,6717,image,PUFIK Interiors & Inspirations,Local save in /data/home-decor,"Scandinavian Cottage,Swedish Cottage,Swedish Home Decor,Swedish Farmhouse,Swedish Style,Swedish Kitchen,Kitchen Black,Swedish House,Cozy Cottage",〚 Уютные шведские коттеджи от Carina Olander 〛 ◾ Фото ◾ Идеи ◾ Дизайн,bc5ab9ee-505e-44f6-92ba-677fe4fdf3e3
home-decor,"6,636 Likes, 141 Comments - The Cottage Journal (@thecottagejournal) on Instagram: “Can you say color?! 😍😍😍 We are loving the cheery vibes that these aqua blue cabinets are g…",1,394,https://i.pinimg.com/originals/8c/17/a2/8c17a257b70780480bb89c3699363144.jpg,6633,image,Sarah Martin,Local save in /data/home-decor,"Diy Kitchen Cabinets,Kitchen Redo,Home Decor Kitchen,New Kitchen,Home Kitchens,Kitchen Remodeling,Aqua Kitchen,Kitchen Counters,Kitchen Islands",The Cottage Journal on Instagram: “Can you say color?! 😍😍😍 We are loving the cheery vibes that these aqua blue cabinets are giving. If you could paint your cabinets any…”,d136f6bc-840d-44f8-bbad-115eb7e6c51e
christmas,"Features: Material:Lint Size:48ｘ18cm Quantity:1 pc Shape:Santa Claus, snowman. Elk Occasion:Christmas Description: 1. Fashion design, high quality 2. Santa Claus, snowman. Elk C…",1,5k,https://i.pinimg.com/originals/b5/7f/21/b57f219fa89c1165b57525b8eae711da.jpg,1706,image,Wear24-7,Local save in /data/christmas,"Merry Christmas To You,Christmas Toys,Great Christmas Gifts,Christmas Snowman,Christmas Ornaments,Holiday,Christmas Party Decorations,Christmas Themes,Decoration Party",Standing Figurine Toys Xmas Santa Claus Snowman Reindeer Figure Plush Dolls Christmas Decorations Ornaments Home Indoor Table Ornaments Christmas Party Tree Hanging Decor Toys Gifts for Kids Friends…,b5c8a1b5-9e90-4522-9bec-2477b698d5b7
christmas,"❤️ ❤️ MERRY CHRISTMAS ❤️ ❤️ ❤️ ❤️ Early Christmas Special:Buy 3 Get 1 Free, Buy 5 Get 2 Free,Deadline November 25. Color:GreenMaterial:Polyvinyl ChlorideItem Dimensions:LxWxH 20…",1,784,https://i.pinimg.com/originals/ef/40/7e/ef407e9568aa46fed4162bd1fd28786e.jpg,1676,image,paupoo,Local save in /data/christmas,"Christmas Hanging Baskets,Christmas Plants,Christmas Wreaths,Christmas Ornaments,Merry Christmas,Christmas Sale,Christmas Porch Ideas,Hanging Christmas Lights,Christmas Island",PAUPOO™ Pre-lit Artificial Christmas Hanging Basket - Flocked with Mixed Decorations and White LED Lights - Frosted Berry BUY 5 GET 2 FREE(7PACKS),3ed92c2d-9cca-4ccf-ac25-44a9d8bec919
christmas,"Christmas decorating ideas for porches. Beautiful holiday decor ideas for front porches both small and large. Outdoor decorations like sleds, lanterns, Christmas trees, wreaths,…",1,46k,https://i.pinimg.com/videos/thumbnails/originals/40/83/f5/4083f5b4971bf235f89a4784ab87271e.0000001.jpg,2482,video,"Life on Summerhill | Home, Holiday Decor & DIY Website",Local save in /data/christmas,"Diy Christmas Decorations For Home,Farmhouse Christmas Decor,Christmas Home,Christmas Holidays,Christmas Front Porches,How To Decorate For Christmas,Christmas Porch Ideas,Christmas Decorating Ideas,Large Outdoor Christmas Decorations",FORNT PORCH CHRISTMAS DECORATING IDEAS,08604f20-fa17-4b9a-9949-781717eca6cd
home-decor,Description The Calligraphy Collection by Allan Copley Designs is meticulously crafted with great attention to detail. The Espresso on Kulin finish with Brushed Stainless Steel…,1,137,https://i.pinimg.com/originals/5f/4f/9f/5f4f9ff6154aba517a852ceb967e2c11.jpg,6521,image,"Your Home, Reimagined LLC",Local save in /data/home-decor,"Glass Top End Tables,End Table Sets,Sofa End Tables,End Tables With Storage,Side Tables,Metal Furniture,Rustic Furnit

# Data Cleaning for `pin` Data
Replaces null values and cleans specfic columns

In [0]:
# replace matching values with null
def replace_matching_values_with_null(df, col_name, match_value):

    return df.withColumn(col_name, when(col(col_name).like(match_value), None).otherwise(col(col_name)))

In [0]:
# dicitonary of columns and values to replace
columns_and_replacements = {
    "description": "No description available%",
    "follower_count": "User Info Error",
    "image_src": "Image src error.",
    "poster_name": "User Info Error",
    "tag_list": "N,o, ,T,a,g,s, ,A,v,a,i,l,a,b,l,e",
    "title": "No Title Data Available"
}

# Loop through the dictionary, calling the function for each column and value
for column_name, match_value in columns_and_replacements.items():
    df_pin = replace_matching_values_with_null(df_pin, column_name, match_value)

## **Data Transformation for `pin` Data**
The following transformations include replacing string indicators in `follower_count` and casting columns to appropriate data types.

In [0]:
# Replace 'K' and 'M' with '000' and '000000'
df_pin = df_pin.withColumn("follower_count", regexp_replace("follower_count", "k", "000"))
df_pin = df_pin.withColumn("follower_count", regexp_replace("follower_count", "M", "000000"))

In [0]:
# Cast the specified numeric columns to the correct data type (int)
numeric_fields = [ "downloaded", "follower_count", "index"]
for field in numeric_fields:
    df_pin = df_pin.withColumn(field, col(field).cast("int"))

In [0]:
# Remove the "Local save in " from the "save_location" column
df_pin = df_pin.withColumn("save_location", regexp_replace("save_location", "Local save in ", ""))

In [0]:
# Rename 'index' to 'ind'
df_pin = df_pin.withColumnRenamed("index", "ind")

In [0]:

# Reorder the columns
df_pin = df_pin.select("ind", "unique_id", "title", "description", "follower_count", 
                       "poster_name", "tag_list", "is_image_or_video", 
                       "image_src", "save_location", "category")

In [0]:
display(df_pin)

ind,unique_id,title,description,follower_count,poster_name,tag_list,is_image_or_video,image_src,save_location,category
6717,bc5ab9ee-505e-44f6-92ba-677fe4fdf3e3,〚 Уютные шведские коттеджи от Carina Olander 〛 ◾ Фото ◾ Идеи ◾ Дизайн,"Традиционные шведские коттеджи, обычно с красным фасадом — это настоящее воплощением идеального зимнего уюта. Они обычно оформлены очень просто и ✌PUFIK. Beautiful Interiors. On…",136000,PUFIK Interiors & Inspirations,"Scandinavian Cottage,Swedish Cottage,Swedish Home Decor,Swedish Farmhouse,Swedish Style,Swedish Kitchen,Kitchen Black,Swedish House,Cozy Cottage",image,https://i.pinimg.com/originals/32/eb/72/32eb72e4fd8654c115a64528bd1f34b4.png,/data/home-decor,home-decor
6633,d136f6bc-840d-44f8-bbad-115eb7e6c51e,The Cottage Journal on Instagram: “Can you say color?! 😍😍😍 We are loving the cheery vibes that these aqua blue cabinets are giving. If you could paint your cabinets any…”,"6,636 Likes, 141 Comments - The Cottage Journal (@thecottagejournal) on Instagram: “Can you say color?! 😍😍😍 We are loving the cheery vibes that these aqua blue cabinets are g…",394,Sarah Martin,"Diy Kitchen Cabinets,Kitchen Redo,Home Decor Kitchen,New Kitchen,Home Kitchens,Kitchen Remodeling,Aqua Kitchen,Kitchen Counters,Kitchen Islands",image,https://i.pinimg.com/originals/8c/17/a2/8c17a257b70780480bb89c3699363144.jpg,/data/home-decor,home-decor
1706,b5c8a1b5-9e90-4522-9bec-2477b698d5b7,Standing Figurine Toys Xmas Santa Claus Snowman Reindeer Figure Plush Dolls Christmas Decorations Ornaments Home Indoor Table Ornaments Christmas Party Tree Hanging Decor Toys Gifts for Kids Friends…,"Features: Material:Lint Size:48ｘ18cm Quantity:1 pc Shape:Santa Claus, snowman. Elk Occasion:Christmas Description: 1. Fashion design, high quality 2. Santa Claus, snowman. Elk C…",5000,Wear24-7,"Merry Christmas To You,Christmas Toys,Great Christmas Gifts,Christmas Snowman,Christmas Ornaments,Holiday,Christmas Party Decorations,Christmas Themes,Decoration Party",image,https://i.pinimg.com/originals/b5/7f/21/b57f219fa89c1165b57525b8eae711da.jpg,/data/christmas,christmas
1676,3ed92c2d-9cca-4ccf-ac25-44a9d8bec919,PAUPOO™ Pre-lit Artificial Christmas Hanging Basket - Flocked with Mixed Decorations and White LED Lights - Frosted Berry BUY 5 GET 2 FREE(7PACKS),"❤️ ❤️ MERRY CHRISTMAS ❤️ ❤️ ❤️ ❤️ Early Christmas Special:Buy 3 Get 1 Free, Buy 5 Get 2 Free,Deadline November 25. Color:GreenMaterial:Polyvinyl ChlorideItem Dimensions:LxWxH 20…",784,paupoo,"Christmas Hanging Baskets,Christmas Plants,Christmas Wreaths,Christmas Ornaments,Merry Christmas,Christmas Sale,Christmas Porch Ideas,Hanging Christmas Lights,Christmas Island",image,https://i.pinimg.com/originals/ef/40/7e/ef407e9568aa46fed4162bd1fd28786e.jpg,/data/christmas,christmas
2482,08604f20-fa17-4b9a-9949-781717eca6cd,FORNT PORCH CHRISTMAS DECORATING IDEAS,"Christmas decorating ideas for porches. Beautiful holiday decor ideas for front porches both small and large. Outdoor decorations like sleds, lanterns, Christmas trees, wreaths,…",46000,"Life on Summerhill | Home, Holiday Decor & DIY Website","Diy Christmas Decorations For Home,Farmhouse Christmas Decor,Christmas Home,Christmas Holidays,Christmas Front Porches,How To Decorate For Christmas,Christmas Porch Ideas,Christmas Decorating Ideas,Large Outdoor Christmas Decorations",video,https://i.pinimg.com/videos/thumbnails/originals/40/83/f5/4083f5b4971bf235f89a4784ab87271e.0000001.jpg,/data/christmas,christmas
6521,dd508c7e-4ff3-4a94-94cb-abc1327c1f58,Allan Copley Designs Calligraphy Square Glass Top End Table in Espresso Finish with Brushed Stainless Steel Accents by Allan Copley Designs - Espresso,Description The Calligraphy Collection by Allan Copley Designs is meticulously crafted with great attention to detail. The Espresso on Kulin finish with Brushed Stainless Steel…,137,"Your Home, Reimagined LLC","Glass Top End Tables,End Table Sets,Sofa End Tables,End Tables With Storage,Side Tables,Metal Furniture,Rustic Furniture,Living Room Furniture,

## **Process `geo` Data**
This section performs transformations on the geo Dataframe, including creating a `coordinates` array and formatting timestamps.

In [0]:
# combine lat and long to create a coordinate column
df_geo = df_geo.withColumn("coordinates", array("latitude", "longitude"))
# drop orginal columns
df_geo = df_geo.drop("latitude", "longitude")
# convert to correct timestamp data type
df_geo=df_geo.withColumn("timestamp", to_timestamp("timestamp"))
# Reorder the columns
df_geo = df_geo.select("ind", "country", "coordinates", "timestamp")

In [0]:
display(df_geo)

ind,country,coordinates,timestamp
9455,British Indian Ocean Territory (Chagos Archipelago),"List(-82.9272, -150.346)",2022-03-15T01:46:32.000+0000
6814,British Indian Ocean Territory (Chagos Archipelago),"List(-86.5675, -149.565)",2022-09-02T11:34:28.000+0000
5111,British Indian Ocean Territory (Chagos Archipelago),"List(-83.7472, 8.65953)",2021-04-01T00:56:57.000+0000
10073,Antarctica (the territory South of 60 deg S),"List(-32.8885, -170.295)",2021-06-29T19:56:04.000+0000
2418,Antarctica (the territory South of 60 deg S),"List(-88.4642, -171.061)",2022-05-27T11:30:59.000+0000
5162,Antarctica (the territory South of 60 deg S),"List(-71.6607, -149.206)",2019-09-27T19:06:43.000+0000
1335,Antarctica (the territory South of 60 deg S),"List(-77.9931, -175.682)",2022-03-19T17:29:42.000+0000
9185,Antarctica (the territory South of 60 deg S),"List(-10.3764, -22.9809)",2019-10-06T18:12:55.000+0000
9335,Antarctica (the territory South of 60 deg S),"List(-88.4642, -171.061)",2020-11-14T23:42:22.000+0000
6749,Antarctica (the territory South of 60 deg S),"List(-88.4642, -171.061)",2018-04-16T07:39:46.000+0000


# **Process `user` Data**


In [0]:
# Create a new column 'user_name' by concatenating first and last names
df_user = df_user.withColumn("user_name ", concat_ws(" ", col("first_name"), col("last_name")))
# drop orginal columns
df_user = df_user.drop("first_name", "last_name")
# convert date_joined to timestamp
df_user=df_user.withColumn("date_joined", to_timestamp("date_joined"))

In [0]:
display(df_user)

age,date_joined,ind,user_name,age_group
27,2016-03-08T13:38:37.000+0000,2015,Christopher Bradshaw,25-35
59,2017-05-12T21:22:17.000+0000,10673,Alexander Cervantes,50+
48,2016-02-27T16:57:44.000+0000,1857,Christopher Hamilton,36-50
45,2016-09-15T06:02:53.000+0000,10020,Christopher Hawkins,36-50
35,2015-10-22T22:42:23.000+0000,2041,Christopher Campbell,25-35
39,2016-06-29T20:43:59.000+0000,6398,Christina Davenport,36-50
20,2015-10-23T04:13:23.000+0000,3599,Alexandria Alvarado,18-24
20,2015-10-23T04:13:23.000+0000,4256,Alexandria Alvarado,18-24
44,2016-12-18T16:05:39.000+0000,1901,Michelle Richardson,36-50
20,2015-10-23T04:13:23.000+0000,3831,Alexandria Alvarado,18-24


T6 For each country find the user with the most followers.

In [0]:
from pyspark.sql import functions as F

# joins df_pin and df_geo on ind column
df_pin_geo = df_pin.join(df_geo, on="ind")
# Casting follower_count to int data type
df_pin_geo = df_pin_geo.withColumn("follower_count", F.col("follower_count").cast("int"))

# Find the user with the most followers for each country, one row of unique value
df_most_followers_per_country = df_pin_geo.groupBy("country", "poster_name").agg(F.max("follower_count").alias("follower_count"))

# Display the results of qeury
df_most_followers_per_country.show()


+--------------------+--------------------+--------------+
 country| poster_name|follower_count|
+--------------------+--------------------+--------------+
 Hungary| Pandora| 848|
 Venezuela|My Poppet - A Cra...| 100000|
 Bahrain| Decor Home Ideas| 144000|
 Uganda| Lauren McBride| 84000|
Bouvet Island (Bo...|SelfMadeLadies | ...| 34000|
 Cape Verde|RT Lifestyle Maga...| 43000|
 Congo|Lindsay Ann Learn...| 6000|
 Burkina Faso| Dare to Cultivate| 7000|
 Benin| Prettyluhhazel| 3000|
 Guyana| Sarah Martin| 394|
 Algeria|The Portable Wife...| 5000|
 Belarus| Ricky Lee| 709|
 Belize|Easy Kids Crafts ...| 267000|
 Azerbaijan|Early Learning Ideas| 50000|
Bouvet Island (Bo...|I Heart Crafty Th...| 276000|
 Austria|Wanderlust Chloe ...| 10000|
Antarctica (the t...| HikenDip| 501000|
 Israel| Leonid Afremov| 1000|
 Albania|ArtsyCraftsyMom |...| 229000|
 Afghanistan| Walmart| 2000000|
+--------------------+--------------------+--------------+
only showing top 20 rows

In [0]:
display(df_most_followers_per_country)

country,poster_name,follower_count
Hungary,Pandora,848
Venezuela,"My Poppet - A Craft, Travel & Food Blog for Colourful Living",100000
Bahrain,Decor Home Ideas,144000
Uganda,Lauren McBride,84000
Bouvet Island (Bouvetoya),"SelfMadeLadies | Law of Attraction, Manifestation & Money Mindset",34000
Cape Verde,"RT Lifestyle Magazine | Travel, DIY, Recipes, Fitness & More",43000
Congo,Lindsay Ann Learning | High School English Activities,6000
Burkina Faso,Dare to Cultivate,7000
Benin,Prettyluhhazel,3000
Guyana,Sarah Martin,394


In [0]:

df_country_most_followers = df_most_followers_per_country.orderBy(F.col("follower_count").desc()).limit(1)


df_country_most_followers.select("country", "follower_count").show()


+--------------+--------------+
 country|follower_count|
+--------------+--------------+
American Samoa| 8000000|
+--------------+--------------+

In [0]:
# What is the most popular category people post to based on the following age groups:
df_user = df_user.withColumn("age_group", 
                             F.when((F.col("age") >= 18) & (F.col("age") <= 24), "18-24")
                             .when((F.col("age") >= 25) & (F.col("age") <= 35), "25-35")
                             .when((F.col("age") >= 36) & (F.col("age") <= 50), "36-50")
                             .otherwise("50+"))

In [0]:
df_combined = df_pin.join(df_user, on="ind")

In [0]:

df_category_count = df_combined.groupBy("age_group", "category").agg(F.count("*").alias("category_count"))


In [0]:
df_category_count.show()

+---------+--------------+--------------+
age_group| category|category_count|
+---------+--------------+--------------+
 18-24| tattoos| 45|
 36-50| travel| 19|
 25-35| quotes| 11|
 25-35|event-planning| 20|
 18-24| travel| 20|
 36-50| education| 10|
 25-35| art| 21|
 25-35| finance| 23|
 18-24| education| 32|
 50+| quotes| 4|
 36-50| quotes| 17|
 50+| mens-fashion| 9|
 18-24|diy-and-crafts| 32|
 18-24|event-planning| 19|
 25-35| education| 11|
 25-35| mens-fashion| 22|
 36-50| art| 12|
 36-50| christmas| 14|
 25-35|diy-and-crafts| 21|
 36-50|event-planning| 11|
+---------+--------------+--------------+
only showing top 20 rows